# Table of Contents
 <p><div class="lev2 toc-item"><a href="#Reference" data-toc-modified-id="Reference-01"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>Reference</a></div><div class="lev2 toc-item"><a href="#XGBoost-for-Classification" data-toc-modified-id="XGBoost-for-Classification-02"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>XGBoost for Classification</a></div><div class="lev3 toc-item"><a href="#Get-input-data" data-toc-modified-id="Get-input-data-021"><span class="toc-item-num">0.2.1&nbsp;&nbsp;</span>Get input data</a></div><div class="lev3 toc-item"><a href="#Add-g_i-and-h_i-to-data" data-toc-modified-id="Add-g_i-and-h_i-to-data-022"><span class="toc-item-num">0.2.2&nbsp;&nbsp;</span>Add $g_i$ and $h_i$ to data</a></div><div class="lev3 toc-item"><a href="#Similarity-Computation" data-toc-modified-id="Similarity-Computation-023"><span class="toc-item-num">0.2.3&nbsp;&nbsp;</span>Similarity Computation</a></div><div class="lev3 toc-item"><a href="#Find-Optimal-Classification-Criteria-(cutoff-value-+-feature)" data-toc-modified-id="Find-Optimal-Classification-Criteria-(cutoff-value-+-feature)-024"><span class="toc-item-num">0.2.4&nbsp;&nbsp;</span>Find Optimal Classification Criteria (cutoff value + feature)</a></div><div class="lev4 toc-item"><a href="#Some-Sanity-Checks" data-toc-modified-id="Some-Sanity-Checks-0241"><span class="toc-item-num">0.2.4.1&nbsp;&nbsp;</span>Some Sanity Checks</a></div><div class="lev2 toc-item"><a href="#Return-a-tree-by-gradient-boost,-and-data" data-toc-modified-id="Return-a-tree-by-gradient-boost,-and-data-03"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>Return a tree by gradient boost, and data</a></div><div class="lev3 toc-item"><a href="#Build-First-Tree" data-toc-modified-id="Build-First-Tree-031"><span class="toc-item-num">0.3.1&nbsp;&nbsp;</span>Build First Tree</a></div><div class="lev3 toc-item"><a href="#Build-Second-Tree" data-toc-modified-id="Build-Second-Tree-032"><span class="toc-item-num">0.3.2&nbsp;&nbsp;</span>Build Second Tree</a></div><div class="lev3 toc-item"><a href="#Put-Together" data-toc-modified-id="Put-Together-033"><span class="toc-item-num">0.3.3&nbsp;&nbsp;</span>Put Together</a></div>

## Reference

In this Jupyter, we follow the article in the below blog and replicate their results:

- use their data as input to run regression
- build tree with recursion
- print out tree with BFS (level by level)
- as an illustration, the hyper-parameters are quite simple

https://img-blog.csdn.net/2018031022534893?watermark/2/text/aHR0cDovL2Jsb2cuY3Nkbi5uZXQvcXFfMjIyMzg1MzM=/font/5a6L5L2T/fontsize/400/fill/I0JBQkFCMA==/dissolve/70

Tianqi Chen's XGBoost Paper

https://arxiv.org/pdf/1603.02754.pdf

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

## XGBoost for Classification

https://blog.csdn.net/qq_22238533/article/details/79477547

Let's follow TianQi's convention

$L^{(t)}(q) = -\frac{1}{2} \sum_{j=1}^T \frac{(\sum_{i \in I_j} g_i)^2}{\sum_{i \in I_j} h_i + \lambda}$

Or

$L^{(t)}(q) = -\frac{1}{2} \sum_{j=1}^T (\sum_{i \in I_j} g_i)^2/ ( \sum_{i \in I_j} h_i + \lambda)$

Here, $T$ is the total number of leaves, or regions in the tree. $i$ is the index of each data. For regression

$g_i = (y_i - \hat{y}_i)$

$h_i = \text{"previous probabilities"*(1-"previous probabilities")}$

### Get input data

In [ ]:
def compute_g(df):
    '''
    Compute 1st derivative with previous probability and target value
    '''
    g = - (df['y'] - df['prev_p'])    
    return g
def compute_h(df):
    '''
    Compute 2nd derivative with previous prob
    '''
    h = df['prev_p'] * (1 - df['prev_p'])
    return h


In [ ]:
df = pd.read_csv("../../XGBoost_Input.csv")
# Initialize the prediction with prob=0.5
base_score = 0.5
df['prev_p'] = base_score

# Hyper-parameters
LAMBDA = 1
MAX_DEPTH = 3
ETA = 0.1 # the learning rate
GAMMA = 0


### Add $g_i$ and $h_i$ to data

In [ ]:
df['g'] = compute_g(df)
df['h'] = compute_h(df)

### Similarity Computation

Note, for each observation we have $L_i$ and $L_{all} = \sum_i^N L_{i}$

$ L(y_i, p_i) = - y_i \ln p_i - (1-y_i) \ln (1-p_i)$

$ L(y_i, \hat{y}_i)  = y_i \ln (1+e^{-\hat{y}_i}) + (1-y_i) \ln (1+e^{\hat{y}_i})$

Then it's 1st and 2nd order derivative in terms of $\hat{y}_i$:

$L^\prime = g_i = \partial_{\hat{y}(t-1)} L(y_i, \hat{y}^{(t-1)}) =  - (y_i - p_i)$

$L^{\prime \prime} = h_i = \partial^2_{\hat{y}(t-1)} L(y_i, \hat{y}^{(t-1)}) = p_i (1-p_i)$

In [ ]:
def leaf_similarity(df, lambda_0):
    '''
    sum(gi)/(sum(hi) + lambda_0)
    '''
    #print(df)
    numerator = pow(df['g'].sum(), 2)
    denom = (df['h'].sum() + lambda_0)
    #denom = (pow(df['h'].sum(), 2) + lambda_0)
    return numerator/float(denom)

### Find Optimal Classification Criteria (cutoff value + feature)

**The Gamma check, and cover check has not been added yet!!**

In [ ]:
def find_best_classifer(df, col='x', lambda_0=0, verbose=False):
    '''
    This function, check all the values of each feature (x1, x2), and check the similarity score
    with cutoff (left leaves < cutoff; right leveas >= cutoff)
    Specifically, if x1: [1, 2, 3, 4, 5]
    x2: [10, 11, 12, 13, 14]
    The cutoff is any one of the below that gives the greatest gain in similarity:

    cutoff 1: x1(< 1, >= 1)
    cutoff 2: x1(< 2, >= 2)
    cutoff 3: x1(< 3, >= 3)
    cutoff 4: x1(< 4, >= 4)
    cutoff 5: x1(< 5, >= 5)
    cutoff 6: x2(< 10, >= 10)
    cutoff 7: x2(< 11, >= 11)
    cutoff 8: x2(< 12, >= 12)
    cutoff 9: x2(< 13, >= 13)
    cutoff 10: x2(< 14, >= 14)

    return: a dictionary contains information, best cutoff, similarity of left & right leaf, 
    the data in left & right leaf
    '''
    df = df.sort_values([col], ascending=[1])
    df = df.reset_index(drop=True)
    clssifications = []
    max_classifier = {}
    no_split = leaf_similarity(df, lambda_0)
    ans_gain = float('-inf')
    cutoffs = sorted(df[col].unique()) # add this line to replicate the result
    # sometimes, there are two classifers, however, we sort them and select the later
    for cutoff in cutoffs:
        df_L = df.loc[df[col] < cutoff]
        df_R = df.loc[np.invert(df[col] < cutoff)]
        score_L = leaf_similarity(df_L, lambda_0)
        score_R = leaf_similarity(df_R, lambda_0)
        #cutoff = (df_L.iloc[-1][col] + df_R.iloc[0][col])/2.0
        if verbose: print(cutoff)
        gain = score_L + score_R - no_split
        if verbose: print("original = {}, socre L={}, score R={}, gain={}".format(no_split, score_L, score_R, gain))
        if gain >= ans_gain:
            ans_gain = gain
            max_classifier = {'cutoff':cutoff, 'left':score_L, 'right':score_R, 'gain':gain, 'left_data':df_L, 'right_data':df_R}
        #break
    return max_classifier

def find_feature_classifier(df, feature_cols, lambda_0, verbose=False):
    best_cutoff_of_features = {"gain":float('-inf')}
    for col in feature_cols:
        temp = find_best_classifer(df, col=col, lambda_0=lambda_0, verbose=verbose)
        if temp['gain'] >= best_cutoff_of_features['gain']:
            best_cutoff_of_features = temp
            best_cutoff_of_features['feature'] = col
            
    return best_cutoff_of_features

def split_by_cutoff(df, cutoff_info):
    cutoff = cutoff_info['cutoff']
    feature = cutoff_info['feature']
    mask = df[feature] < cutoff
    df_L, df_R = df.loc[mask], df.loc[~mask]
    return df_L, df_R

def compute_weight(df, lambda_0=1, eta=0.1):
    '''
    Compute weight of a region, note, eta is the learning rate and is applied to scale the weight
    '''
    return -eta*df['g'].sum()/(df['h'].sum()+lambda_0)

#### Some Sanity Checks

We will use a function to recursively build the tree, however, in this section, we check some cutoff points for the given inputs.

In [ ]:
level_1_cutoff_info = find_feature_classifier(df, ['x1', 'x2'], lambda_0=1)

In [ ]:
df_L = [None] * MAX_DEPTH
df_R = [None] * MAX_DEPTH

In [ ]:
df_L[1], df_R[1] = split_by_cutoff(df, cutoff_info=level_1_cutoff_info)

In [ ]:
df_L[1]

In [ ]:
compute_weight(df_R[1])

In [ ]:
df_R[1] # Right leaf has only one data, stop here

In [ ]:
# depth = 1

# # check the similarity of one specific column
# # 
# check1 = find_best_classifer(df_L[1], col='x1', lambda_0=LAMBDA, verbose=False)
# print(check1['gain'], check1['cutoff']) # when we look at x1 column, and use x1 < 3, then we get maximum gain which is 0.2539

# # check the similarity of one specific column
# # 
# check1 = find_best_classifer(df_L[1], col='x2', lambda_0=LAMBDA, verbose=False)
# print(check1['gain'], check1['cutoff']) # when we look at x1 column, and use x1 < 3, then we get maximum gain which is 0.2539

In [ ]:
level_2_cutoff_info = find_feature_classifier(df_L[1], ['x1', 'x2'], lambda_0=1)

In [ ]:
level_2_cutoff_info

In [ ]:
# The depth = 2

In [ ]:
df_L[2], df_R[2] = split_by_cutoff(df_L[1], cutoff_info=level_2_cutoff_info)

In [ ]:
find_feature_classifier(df_R[2], ['x1'], lambda_0=1, verbose=True)

## Return a tree by gradient boost, and data


In [ ]:
class TreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

def buildTree(df, height=0):
    '''
    Build a tree with df data with differen features;
    if the tree depth is reached, then stop split;
    for all the internal nodes, save cutoff + feature as the val;
    for leaves, save weight as the val
    '''
    if df.shape[0]<=1 or height >= MAX_DEPTH:
        weight = compute_weight(df)
        root = TreeNode()
        root.val = {'weight':weight, 'depth':height}
        return root
    
    curr_node = TreeNode()
    cutoff_info = find_feature_classifier(df, feature_cols=['x1', 'x2'], lambda_0=1)
    curr_node.val = {"cutoff":cutoff_info['cutoff'], 'depth':height, 'feature':cutoff_info['feature'] }
    df_L, df_R = split_by_cutoff(df, cutoff_info=cutoff_info)
    curr_node.left = buildTree(df_L.copy(), height+1)
    curr_node.right = buildTree(df_R.copy(), height+1)
    
    return curr_node
    
def traversal_bfs(root: TreeNode):
    '''
    BFS to print out the tree by level
    '''
    from collections import deque
    result = []
    if root is None:
        return []
    queue = deque()
    queue.append(root)
    depth = 0
    while queue:
        current_level = []
        length = len(queue)
        for _ in range(length):
            node = queue.popleft()
            current_level.append(node.val)
            if node.left is not None: queue.append(node.left)
            if node.right is not None: queue.append(node.right)
        result.append(current_level)
    return result

def update_prev(df, root):
    df2 = df.copy()
    f_prev = np.log(df2['prev_p']/(1-df2['prev_p']))
    f_curr = df2.apply(f_tree, root=root, axis=1)    
    df2['prev_p'] = 1-1/(1+np.exp(f_prev + f_curr))
    return df2

def f_tree(x, root):
    if 'weight' in root.val:
        return root.val['weight']
    feature = root.val['feature']
    if x[feature] < root.val['cutoff']:
        return f_tree(x, root.left)
    else:
        return f_tree(x, root.right)

def update_g_h_and_rebuild(df):
    '''
    df['prev_p'] --> add two df['h'], df['g']
    return df, root
    '''
    df['g'] = compute_g(df)
    df['h'] = compute_h(df)
    root = buildTree(df)
    return df, root

### Build First Tree

In [ ]:
root = buildTree(df)

In [ ]:
traversal_bfs(root)

Note:
Our base value $p=0.5 \rightarrow \hat{y}_0 = 0$ 

In addition, $p$ is a sigmod mapping of our weights

$p = \frac{1}{1+\exp (\sum_{i=0}^M\hat{y}_i)}$

The weights that we assigned to the leaves are in fact $\hat{y}$.

In [ ]:
df2 = df.copy()

In [ ]:
df2.head(3)

In [ ]:
df2 = update_prev(df, root)

In [ ]:
df2 = df2.set_index('ID')

In [ ]:
with pd.option_context('display.float_format', '{:0.6f}'.format):
    print(df2)

### Build Second Tree

In [ ]:
df2, root2 = update_g_h_and_rebuild(df2)

In [ ]:
traversal_bfs(root2)

### Put Together

In [ ]:
df = pd.read_csv("../../XGBoost_Input.csv")
df = df.set_index('ID')
df['prev_p'] = base_score

LAMBDA = 1
MAX_DEPTH = 3
ETA = 0.1 # the learning rate
GAMMA = 0
base_score = 0.5
for M_trees in range(1, 10):
#M_trees = 2
    for i in range(0, M_trees):
        df, root = update_g_h_and_rebuild(df)
        df = update_prev(df, root)
    df['hat(y)'] = (df['prev_p'] > 0.5).astype(int)
    print("M(# of tree) = {}".format(M_trees))
    print("Precision:",(df['y']==df['hat(y)']).sum()/df.shape[0])